In [13]:
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import numpy as np

from linearmodels.panel import PanelOLS
import datetime as dt

# Characteristics OLS

Professor said we should start with a regular OLS Regression.
I quickly looked into this and think a random effects panel regression would probably fit our data better (still need to read more into this)
So probably we can first do a regular OLS and than compare it to the random effects panel regression. Just some initial thoughts though!!!
Here is some intial code to run them:


In [14]:
#decide which df to use
df = pd.read_csv("Dataframes/characteristics.csv")
#df = df.loc[:, ["Instrument", "Date", "Earnings Per Share - Actual Surprise", "Revenue - Actual", "Net Income after Tax"]]

df

,Instrument,Date,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,POOL.OQ,2022-10-01,4.196,1.615339e+09,1.406160e+10,1.259813e+10,2.383926,-9.164375,238539.0,-1
1,POOL.OQ,2022-07-01,1.503,2.055818e+09,1.556187e+10,1.405795e+10,2.749481,-21.661570,185584.0,-1
2,POOL.OQ,2022-04-01,34.342,1.412650e+09,1.843682e+10,1.696712e+10,3.422880,-25.160664,290897.0,-1
3,POOL.OQ,2022-01-01,40.267,1.035557e+09,2.384882e+10,2.268979e+10,4.504382,30.489705,82349.0,-1
4,POOL.OQ,2021-10-01,17.194,1.411448e+09,1.769954e+10,1.742020e+10,3.684400,-5.130322,157740.0,-1
...,...,...,...,...,...,...,...,...,...,...
19332,AVY.N,2014-01-01,1.471,1.583900e+09,5.553068e+09,4.877168e+09,0.920659,16.003852,157761.0,-1
19333,AVY.N,2013-10-01,8.095,1.504900e+09,5.029514e+09,4.273414e+09,0.903176,2.467620,236291.0,-1
19334,AVY.N,2013-07-01,1.068,1.552300e+09,5.436229e+09,4.258229e+09,0.906078,-0.054218,0.0,-1
19335,AVY.N,2013-04-01,2.482,1.498900e+09,5.459128e+09,4.309428e+09,0.895787,24.148726,0.0,-1


#### OLS Regression

In [15]:
#identifying outliers and replacing them with NA
summary_stats = df["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats.loc['25%']
Q3 = summary_stats.loc['75%']
IQR = Q3 - Q1
threshold = 7 #1.5 is standard threshold but we still want to keep enough variation in the data so setting threshol higher here
surprise_outliers_removed = df["Earnings Per Share - Actual Surprise"].loc[~((df["Earnings Per Share - Actual Surprise"] < (Q1 - threshold * IQR)) | (df["Earnings Per Share - Actual Surprise"] > (Q3 + threshold * IQR)))]
df_accuracy_new = df.copy()
df_accuracy_new["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed
df_accuracy_new = df_accuracy_new.dropna(subset=["Earnings Per Share - Actual Surprise"])
#df_accuracy_new["Recommendation - Mean (1-5)"] = df_accuracy_new["Recommendation - Mean (1-5)"].fillna(0)
df_accuracy_new = df_accuracy_new.dropna()
df_accuracy_new.value_counts("loss firm status")


loss firm status
-1    17281
 1     1271
 0       14
dtype: int64

In [16]:
df_accuracy_new

,Instrument,Date,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,POOL.OQ,2022-10-01,4.196,1.615339e+09,1.406160e+10,1.259813e+10,2.383926,-9.164375,238539.0,-1
1,POOL.OQ,2022-07-01,1.503,2.055818e+09,1.556187e+10,1.405795e+10,2.749481,-21.661570,185584.0,-1
2,POOL.OQ,2022-04-01,34.342,1.412650e+09,1.843682e+10,1.696712e+10,3.422880,-25.160664,290897.0,-1
3,POOL.OQ,2022-01-01,40.267,1.035557e+09,2.384882e+10,2.268979e+10,4.504382,30.489705,82349.0,-1
4,POOL.OQ,2021-10-01,17.194,1.411448e+09,1.769954e+10,1.742020e+10,3.684400,-5.130322,157740.0,-1
...,...,...,...,...,...,...,...,...,...,...
19332,AVY.N,2014-01-01,1.471,1.583900e+09,5.553068e+09,4.877168e+09,0.920659,16.003852,157761.0,-1
19333,AVY.N,2013-10-01,8.095,1.504900e+09,5.029514e+09,4.273414e+09,0.903176,2.467620,236291.0,-1
19334,AVY.N,2013-07-01,1.068,1.552300e+09,5.436229e+09,4.258229e+09,0.906078,-0.054218,0.0,-1
19335,AVY.N,2013-04-01,2.482,1.498900e+09,5.459128e+09,4.309428e+09,0.895787,24.148726,0.0,-1


In [17]:
rescale = df_accuracy_new
#rescale["Earnings Per Share - Actual Surprise"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share - Actual Surprise"]).reshape(-1,1))
rescale["Revenue - Actual"] = MinMaxScaler().fit_transform(np.array(rescale["Revenue - Actual"]).reshape(-1,1))
rescale["Enterprise Value"] = MinMaxScaler().fit_transform(np.array(rescale["Enterprise Value"]).reshape(-1,1))
rescale["Market Capitalization"] = MinMaxScaler().fit_transform(np.array(rescale["Market Capitalization"]).reshape(-1,1))
rescale["Enterprise Value To Sales (Daily Time Series Ratio)"] = MinMaxScaler().fit_transform(np.array(rescale["Enterprise Value To Sales (Daily Time Series Ratio)"]).reshape(-1,1))
rescale["3 Month Total Return"] = MinMaxScaler().fit_transform(np.array(rescale["3 Month Total Return"]).reshape(-1,1))
rescale["Volume"] = MinMaxScaler().fit_transform(np.array(rescale["Volume"]).reshape(-1,1))

rescale

,Instrument,Date,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,POOL.OQ,2022-10-01,4.196,0.065637,0.031045,0.004337,0.029064,0.235367,0.001361,-1
1,POOL.OQ,2022-07-01,1.503,0.068358,0.031540,0.004842,0.029495,0.195817,0.001059,-1
2,POOL.OQ,2022-04-01,34.342,0.064385,0.032488,0.005848,0.030290,0.184744,0.001660,-1
3,POOL.OQ,2022-01-01,40.267,0.062055,0.034274,0.007826,0.031566,0.360861,0.000470,-1
4,POOL.OQ,2021-10-01,17.194,0.064377,0.032245,0.006004,0.030598,0.248134,0.000900,-1
...,...,...,...,...,...,...,...,...,...,...
19332,AVY.N,2014-01-01,1.471,0.065443,0.028238,0.001667,0.027338,0.315017,0.000900,-1
19333,AVY.N,2013-10-01,8.095,0.064955,0.028065,0.001459,0.027317,0.272179,0.001348,-1
19334,AVY.N,2013-07-01,1.068,0.065247,0.028199,0.001453,0.027321,0.264198,0.000000,-1
19335,AVY.N,2013-04-01,2.482,0.064917,0.028207,0.001471,0.027308,0.340793,0.000000,-1


In [18]:
X_var_names = ["Revenue - Actual","Enterprise Value","Market Capitalization","loss firm status", "Enterprise Value To Sales (Daily Time Series Ratio)", "3 Month Total Return", "Volume"]
y = rescale["Earnings Per Share - Actual Surprise"]
#X = df_accuracy_new[['Revenue - Actual', 'Net Income after Tax']]
X = rescale[X_var_names]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     OLS Regression Results                                     
================================================================================================
Dep. Variable:     Earnings Per Share - Actual Surprise   R-squared:                       0.014
Model:                                              OLS   Adj. R-squared:                  0.013
Method:                                   Least Squares   F-statistic:                     36.71
Date:                                  Sat, 25 Feb 2023   Prob (F-statistic):           2.14e-51
Time:                                          10:33:15   Log-Likelihood:                -78897.
No. Observations:                                 18566   AIC:                         1.578e+05
Df Residuals:                                     18558   BIC:                         1.579e+05
Df Model:                                             7                                         
Covariance Type:                              nonrobust                                         
=======================================================================================================================
                                                          coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------
const                                                  -4.5379      0.907     -5.004      0.000      -6.315      -2.761
Revenue - Actual                                       -0.7004      2.185     -0.321      0.749      -4.983       3.582
Enterprise Value                                       13.8044     11.367      1.214      0.225      -8.476      36.085
Market Capitalization                                  -5.2955     11.053     -0.479      0.632     -26.961      16.370
loss firm status                                       -0.0279      0.248     -0.113      0.910      -0.513       0.457
Enterprise Value To Sales (Daily Time Series Ratio)   -14.9995     10.118     -1.482      0.138     -34.832       4.833
3 Month Total Return                                   41.2851      2.656     15.546      0.000      36.080      46.490
Volume                                                 14.0347      7.904      1.776      0.076      -1.458      29.528
==============================================================================
Omnibus:                     3744.779   Durbin-Watson:                   1.658
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            33491.949
Skew:                           0.717   Prob(JB):                         0.00
Kurtosis:                       9.422   Cond. No.                         175.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### OLS Pooled Regression

In [19]:
df_accuracy_new['Date'] = pd.to_datetime(df_accuracy_new['Date'], infer_datetime_format=True)
df_accuracy_new.dtypes

Instrument                                                     object
Date                                                   datetime64[ns]
Earnings Per Share - Actual Surprise                          float64
Revenue - Actual                                              float64
Enterprise Value                                              float64
Market Capitalization                                         float64
Enterprise Value To Sales (Daily Time Series Ratio)           float64
3 Month Total Return                                          float64
Volume                                                        float64
loss firm status                                                int64
dtype: object

In [20]:
df_accuracy_new.set_index(["Instrument", "Date"], inplace=True)
df_accuracy_new

Earnings Per Share - Actual Surprise  Revenue - Actual  \
Instrument Date                                                                 
POOL.OQ    2022-10-01                                 4.196          0.065637   
           2022-07-01                                 1.503          0.068358   
           2022-04-01                                34.342          0.064385   
           2022-01-01                                40.267          0.062055   
           2021-10-01                                17.194          0.064377   
...                                                     ...               ...   
AVY.N      2014-01-01                                 1.471          0.065443   
           2013-10-01                                 8.095          0.064955   
           2013-07-01                                 1.068          0.065247   
           2013-04-01                                 2.482          0.064917   
           2013-01-01                                11.178          0.065123   

                       Enterprise Value  Market Capitalization  \
Instrument Date                                                  
POOL.OQ    2022-10-01          0.031045               0.004337   
           2022-07-01          0.031540               0.004842   
           2022-04-01          0.032488               0.005848   
           2022-01-01          0.034274               0.007826   
           2021-10-01          0.032245               0.006004   
...                                 ...                    ...   
AVY.N      2014-01-01          0.028238               0.001667   
           2013-10-01          0.028065               0.001459   
           2013-07-01          0.028199               0.001453   
           2013-04-01          0.028207               0.001471   
           2013-01-01          0.027862               0.001166   

                       Enterprise Value To Sales (Daily Time Series Ratio)  \
Instrument Date                                                              
POOL.OQ    2022-10-01                                           0.029064     
           2022-07-01                                           0.029495     
           2022-04-01                                           0.030290     
           2022-01-01                                           0.031566     
           2021-10-01                                           0.030598     
...                                                                  ...     
AVY.N      2014-01-01                                           0.027338     
           2013-10-01                                           0.027317     
           2013-07-01                                           0.027321     
           2013-04-01                                           0.027308     
           2013-01-01                                           0.027195     

                       3 Month Total Return    Volume  loss firm status  
Instrument Date                                                          
POOL.OQ    2022-10-01              0.235367  0.001361                -1  
           2022-07-01              0.195817  0.001059                -1  
           2022-04-01              0.184744  0.001660                -1  
           2022-01-01              0.360861  0.000470                -1  
           2021-10-01              0.248134  0.000900                -1  
...                                     ...       ...               ...  
AVY.N      2014-01-01              0.315017  0.000900                -1  
           2013-10-01              0.272179  0.001348                -1  
           2013-07-01              0.264198  0.000000                -1  
           2013-04-01              0.340793  0.000000                -1  
           2013-01-01              0.298034  0.001183                -1  

[18566 rows x 8 columns]

In [21]:
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm
import statsmodels.graphics.tsaplots as tsap
from statsmodels.compat import lzip
from statsmodels.stats.diagnostic import het_white
from matplotlib import pyplot as plt

In [22]:
y_var_name = 'Earnings Per Share - Actual Surprise'
pooled_y=df_accuracy_new[y_var_name]
pooled_X=df_accuracy_new[X_var_names]

In [23]:
pooled_X = sm.add_constant(pooled_X)
pooled_X

const  Revenue - Actual  Enterprise Value  \
Instrument Date                                                    
POOL.OQ    2022-10-01    1.0          0.065637          0.031045   
           2022-07-01    1.0          0.068358          0.031540   
           2022-04-01    1.0          0.064385          0.032488   
           2022-01-01    1.0          0.062055          0.034274   
           2021-10-01    1.0          0.064377          0.032245   
...                      ...               ...               ...   
AVY.N      2014-01-01    1.0          0.065443          0.028238   
           2013-10-01    1.0          0.064955          0.028065   
           2013-07-01    1.0          0.065247          0.028199   
           2013-04-01    1.0          0.064917          0.028207   
           2013-01-01    1.0          0.065123          0.027862   

                       Market Capitalization  loss firm status  \
Instrument Date                                                  
POOL.OQ    2022-10-01               0.004337                -1   
           2022-07-01               0.004842                -1   
           2022-04-01               0.005848                -1   
           2022-01-01               0.007826                -1   
           2021-10-01               0.006004                -1   
...                                      ...               ...   
AVY.N      2014-01-01               0.001667                -1   
           2013-10-01               0.001459                -1   
           2013-07-01               0.001453                -1   
           2013-04-01               0.001471                -1   
           2013-01-01               0.001166                -1   

                       Enterprise Value To Sales (Daily Time Series Ratio)  \
Instrument Date                                                              
POOL.OQ    2022-10-01                                           0.029064     
           2022-07-01                                           0.029495     
           2022-04-01                                           0.030290     
           2022-01-01                                           0.031566     
           2021-10-01                                           0.030598     
...                                                                  ...     
AVY.N      2014-01-01                                           0.027338     
           2013-10-01                                           0.027317     
           2013-07-01                                           0.027321     
           2013-04-01                                           0.027308     
           2013-01-01                                           0.027195     

                       3 Month Total Return    Volume  
Instrument Date                                        
POOL.OQ    2022-10-01              0.235367  0.001361  
           2022-07-01              0.195817  0.001059  
           2022-04-01              0.184744  0.001660  
           2022-01-01              0.360861  0.000470  
           2021-10-01              0.248134  0.000900  
...                                     ...       ...  
AVY.N      2014-01-01              0.315017  0.000900  
           2013-10-01              0.272179  0.001348  
           2013-07-01              0.264198  0.000000  
           2013-04-01              0.340793  0.000000  
           2013-01-01              0.298034  0.001183  

[18566 rows x 8 columns]

In [24]:
pooled_olsr_model = sm.OLS(endog=pooled_y, exog=pooled_X)

In [25]:
pooled_olsr_model_results = pooled_olsr_model.fit()

In [26]:
pooled_olsr_model_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     OLS Regression Results                                     
================================================================================================
Dep. Variable:     Earnings Per Share - Actual Surprise   R-squared:                       0.014
Model:                                              OLS   Adj. R-squared:                  0.013
Method:                                   Least Squares   F-statistic:                     36.71
Date:                                  Sat, 25 Feb 2023   Prob (F-statistic):           2.14e-51
Time:                                          10:33:16   Log-Likelihood:                -78897.
No. Observations:                                 18566   AIC:                         1.578e+05
Df Residuals:                                     18558   BIC:                         1.579e+05
Df Model:                                             7                                         
Covariance Type:                              nonrobust                                         
=======================================================================================================================
                                                          coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------
const                                                  -4.5379      0.907     -5.004      0.000      -6.315      -2.761
Revenue - Actual                                       -0.7004      2.185     -0.321      0.749      -4.983       3.582
Enterprise Value                                       13.8044     11.367      1.214      0.225      -8.476      36.085
Market Capitalization                                  -5.2955     11.053     -0.479      0.632     -26.961      16.370
loss firm status                                       -0.0279      0.248     -0.113      0.910      -0.513       0.457
Enterprise Value To Sales (Daily Time Series Ratio)   -14.9995     10.118     -1.482      0.138     -34.832       4.833
3 Month Total Return                                   41.2851      2.656     15.546      0.000      36.080      46.490
Volume                                                 14.0347      7.904      1.776      0.076      -1.458      29.528
==============================================================================
Omnibus:                     3744.779   Durbin-Watson:                   1.658
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            33491.949
Skew:                           0.717   Prob(JB):                         0.00
Kurtosis:                       9.422   Cond. No.                         175.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""